## 崩溃现场
### 1. 崩溃信息
- 进程名、线程名：崩溃的进程是前台进程还是后台进程，崩溃是不是发生在 UI 线程
- 崩溃堆栈和类型：崩溃是属于 Java 崩溃、Native 崩溃，还是 ANR，对于不同类型的崩溃我们关注的点也不太一样。特别需要看崩溃堆栈的栈顶，看具体崩溃在系统的代码，还是我们自己的代码里面

```
Process Name: 'com.sample.crash'
Thread Name: 'MyThread'

java.lang.NullPointerException
    at ...TestsActivity.crashInJava(TestsActivity.java:275)
```

### 2. 系统信息
- Logcat：包括应用、系统的运行日志。由于系统权限问题，获取到的 Logcat 可能只包含与当前 App 相关的。其中系统的 event logcat 会记录 App 运行的一些基本情况，记录在文件 /system/etc/event-log-tags 中

```
system logcat:
10-25 17:13:47.788 21430 21430 D dalvikvm: Trying to load lib ... 

event logcat:
10-25 17:13:47.788 21430 21430 I am_on_resume_called: 生命周期
10-25 17:13:47.788 21430 21430 I am_low_memory: 系统内存不足
10-25 17:13:47.788 21430 21430 I am_destroy_activity: 销毁 Activty
10-25 17:13:47.888 21430 21430 I am_anr: ANR 以及原因
10-25 17:13:47.888 21430 21430 I am_kill: APP 被杀以及原因
```

- 机型、系统、厂商、CPU、ABI、Linux 版本：会采集多达几十个维度，这对后面讲到寻找共性问题会很有帮助。
- 设备状态：是否 root、是否是模拟器。一些问题是由 Xposed 或多开软件造成，对这部分问题我们要区别对待。

### 3. 内存信息
- 系统剩余内存: 关于系统内存状态，可以直接读取文件 /proc/meminfo。当系统可用内存很小（低于 MemTotal 的 10%）时，OOM、大量 GC、系统频繁自杀拉起等问题都非常容易出现。
- 应用使用内存: 包括 Java 内存、RSS（Resident Set Size）、PSS（Proportional Set Size），我们可以得出应用本身内存的占用大小和分布。PSS 和 RSS 通过 /proc/self/smap 计算，可以进一步得到例如 apk、dex、so 等更加详细的分类统计。
- 虚拟内存: 虚拟内存可以通过 /proc/self/status 得到，通过 /proc/self/maps 文件可以得到具体的分布情况。有时候我们一般不太重视虚拟内存，但是很多类似 OOM、tgkill 等问题都是虚拟内存不足导致的。

```
Name:     com.sample.name   // 进程名
FDSize:   800               // 当前进程申请的文件句柄个数
VmPeak:   3004628 kB        // 当前进程的虚拟内存峰值大小
VmSize:   2997032 kB        // 当前进程的虚拟内存大小
Threads:  600               // 当前进程包含的线程个数
```

### 4. 资源信息
- 文件句柄 fd: 文件句柄的限制可以通过 /proc/self/limits 获得，一般单个进程允许打开的最大文件句柄个数为 1024。但是如果文件句柄超过 800 个就比较危险，需要将所有的 fd 以及对应的文件名输出到日志中，进一步排查是否出现了有文件或者线程的泄漏。

```
opened files count 812:
0 -> /dev/null
1 -> /dev/log/main4 
2 -> /dev/binder
3 -> /data/data/com.crash.sample/files/test.config
...
```

- 线程数: 当前线程数大小可以通过上面的 status 文件得到，一个线程可能就占 2MB 的虚拟内存，过多的线程会对虚拟内存和文件句柄带来压力。根据我的经验来说，如果线程数超过 400 个就比较危险。需要将所有的线程 id 以及对应的线程名输出到日志中，进一步排查是否出现了线程相关的问题。

```
 threads count 412:               
 1820 com.sample.crashsdk                         
 1844 ReferenceQueueD                                             
 1869 FinalizerDaemon   
 ...
```

- JNI: 使用 JNI 时，如果不注意很容易出现引用失效、引用爆表等一些崩溃。我们可以通过 DumpReferenceTables 统计 JNI 的引用表，进一步分析是否出现了 JNI 泄漏等问题。

### 5. 应用信息
- 崩溃场景: 崩溃发生在哪个 Activity 或 Fragment，发生在哪个业务中
- 关键操作路径: 不同于开发过程详细的打点日志，我们可以记录关键的用户操作路径，这对我们复现崩溃会有比较大的帮助。
- 其他自定义信息: 不同的应用关心的重点可能不太一样，比如网易云音乐会关注当前播放的音乐，QQ 浏览器会关注当前打开的网址或视频。

---

## 崩溃分析
### 1. 确定重点
- 确认严重程度: 解决崩溃也要看性价比，我们优先解决 Top 崩溃或者对业务有重大影响，例如启动、支付过程的崩溃。

### 2. 崩溃基本信息
#### java 崩溃
- 比较明显，比如 NullPointerException 是空指针，OutOfMemoryError 是资源不足

#### Native 崩溃
- 需要观察 signal、code、fault addr 等内容，以及崩溃时 Java 的堆栈
- 比较常见的是有 SIGSEGV 和 SIGABRT，前者一般是由于空指针、非法指针造成，后者主要因为 ANR 和调用 abort() 退出所导致。

#### ANR
- 经验是，先看看主线程的堆栈，是否是因为锁等待导致。接着看看 ANR 日志中 iowait、CPU、GC、system server 等信息，进一步确定是 I/O 问题，或是 CPU 竞争问题，还是由于大量 GC 导致卡死。

#### 3. Logcat
- Logcat 一般会存在一些有价值的线索，日志级别是 Warning、Error 的需要特别注意。
- 例如出现 ANR 时，会有 "am_anr"
- App 被杀时，会有 "am_kill"
- 当从一条崩溃日志中无法看出问题的原因，或者得不到有用信息时，不要放弃，建议查看相同崩溃点下的更多崩溃日志。

#### 4. 各个资源情况
- 结合崩溃的基本信息，我们接着看看是不是跟 "内存信息" 有关，是不是跟 "资源信息" 有关。比如是物理内存不足、虚拟内存不足，还是文件句柄 fd 泄漏了。
- 无论是资源文件还是 Logcat，内存与线程相关的信息都需要特别注意，很多崩溃都是由于它们使用不当造成的。

### 2. 查找共性
- 如果使用了上面的方法，还是未找到问题，可以尝试找这类崩溃有什么共性。
- 机型、系统、ROM、厂商、ABI，这些采集到的系统信息都可以作为维度聚合，共性问题例如是不是因为安装了 Xposed，是不是只出现在 x86 的手机，是不是只有三星这款机型，是不是只在 Android 5.0 的系统上

### 3. 尝试复现
- 在稳定的复现路径上面，我们可以采用增加日志或使用 Debugger、GDB 等各种各样的手段或工具做进一步分析。

#### 疑难问题：系统崩溃
- 系统崩溃常常令我们感到非常无助，它可能是某个 Android 版本的 bug，也可能是某个厂商修改 ROM 导致。
- 针对这种疑难问题，可以尝试这样的解决思路

1 查找可能的原因
   - 根据共性看看是不是某个系统版本的问题，还是某个厂商特定的 ROM 问题
   
2 尝试规避
   - 查看可疑的代码调用，是否使用了不恰当的 API，是否可以更换其他的实现方式规避
   
3 Hook 解决
   - 分为 Java Hook 和 Native Hook
   - 案例：发现线上出现一个 Toast 相关的系统崩溃，它只出现在 Android 7.0 的系统中，看起来是在 Toast 显示的时候窗口的 token 已经无效了
   ```
   android.view.WindowManager$BadTokenException: 
   at android.view.ViewRootImpl.setView(ViewRootImpl.java)
   at android.view.WindowManagerGlobal.addView(WindowManagerGlobal.java)
   at android.view.WindowManagerImpl.addView(WindowManagerImpl.java4)
   at android.widget.Toast$TN.handleShow(Toast.java)
   ```
   
   - 为什么 Android 8.0 系统没有问题？查看 8.0 源码
   - 考虑再三，我们决定参考 Android 8.0 的做法，直接 catch 住这个异常
   ```
   try {
      mWM.addView(mView, mParams);
      trySendAccessibilityEvent();
    } catch (WindowManager.BadTokenException e) {
      /* ignore */
    }
   ```
   
   - 95% 以上的崩溃都能解决或者规避，大部分的系统崩溃也是如此

---

## 课后作业
- 通过源码分析。我们发现 TimeoutException 是由系统的 FinalizerWatchdogDaemon 抛出来的
- 寻找可以规避的方法。尝试调用了它的 Stop() 方法，但是线上发现在 Android 6.0 之前会有线程同步问题
- 寻找其他可以 Hook 的点。通过代码的依赖关系，发现一个取巧的 Hook 点

```
java.util.concurrent.TimeoutException: 
         android.os.BinderProxy.finalize() timed out after 10 seconds
at android.os.BinderProxy.destroy(Native Method)
at android.os.BinderProxy.finalize(Binder.java:459)
```

---